In [1]:
# make the screen bigger!
from IPython.display import display, HTML
display(HTML(data=""" <style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 85%; }
    div#maintoolbar-container { width: 99%; } </style> """))


#### import modules
import numpy as np
import arcpy
import os
import sys
from arcpy.sa import *
import pandas as pd
from arcpy import env
import shutil
import numpy.ma as ma
import netCDF4 as nc
import subprocess
import re
import datetime
import matplotlib 
import matplotlib.pyplot as plt
import scipy
from scipy.stats import linregress
import scipy.optimize as opt

# Tired of pink boxes?
pd.options.mode.chained_assignment = None  # default='warn'

# set properties
arcpy.env.overwriteOutput = True # make sure overwrite files is on
# projection definition 
sr_project = arcpy.SpatialReference(32702)   # Project dataset into WGS84

input_folder = os.path.join("..", "input")  


GIS_FOLDER = os.path.join('..', '..', 'Raw_GIS_Data')
#STD_INPUT_FOLDER = os.path.join('..', '..', 'Std_input')

# path to the grid bound
Grid_shp = os.path.join(GIS_FOLDER, 'grid_bound.shp')


In [2]:
def mod_ctrl_file_cellSize(Control_File_Name):
    
    # modify the control file gridsize for the given run   (note this uses dimensions from the rainfall adjustment grid in april)
    with open(os.path.join(input_folder, 'RF_adj_grids', 'rfadj_apr.asc'), 'r') as dims_file:   # open an ASC file and get the dimenstions out of it 
        dimsfile1 = dims_file.read().splitlines(True)
        x_dim = float(re.findall('\d+', dimsfile1[0])[-1])    
        y_dim = float(re.findall('\d+', dimsfile1[1])[-1])
        cel_size = float(re.findall('\d+', dimsfile1[4])[-1])

    with open(os.path.join('.', Control_File_Name), 'r') as fin:   # open file 
        data = fin.read().splitlines(True)
    with open(os.path.join('.', Control_File_Name), 'w') as fout:     # delete first line
        fout.writelines(data[1:])
    new_first = 'GRID {} {} 515000. 8409000. {} '.format(x_dim, y_dim, cel_size)  # new first line 
    with open(os.path.join('.', Control_File_Name), 'r+') as file:                # add in new first line and save file  
        file_data = file.read()
        file. seek(0, 0)
        file. write(new_first + '\n' + file_data)
        
        return x_dim, y_dim, cel_size

# Define Run model function


Changes needed to the function
- Specify individual output folders (no making them, need to make them in the cells below 
- Specify individual control files (all Ctrl files should hang out in the run folder with the swb2.exe
                            
get rid of the calibration stuff at the end
                

In [22]:
def PP_model(Control_File_Name, output_folder, x_dim, y_dim, Direct_infiltration_input_folder):
    print(Control_File_Name)
    
    

    # Parameters
    Desired_files = ['actual_et',  'direct_net_infiltation', 'direct_soil_moisture',
                 'interception', 'net_infiltration', 'rainfall', 'runoff'] # 'delta_soil_storage',  'irrigation', 
    XLLCORNER =      515000.000
    YLLCORNER =      8409000.000
    CELLSIZE  =      cel_size

    # functions
    def create_file_reference( component_name ):
        '''
        This is a simple convenience function that will form a path and filename to a
        given water budget component
        '''
        # specify the prefix, path to SWB2 output, timeframe, and resolution
        #output_path = os.path.join(os.getcwd(), "output")
        #prefix      = '\\'
        
    # pull start and end dates from control file
        with open(os.path.join('.', Control_File_Name), 'r') as fin:   # open file 
            date_find = fin.read().splitlines(True)
        start_year = datetime.datetime.strptime(date_find[-2].split(" ")[1], "%m/%d/%Y").strftime("%Y-%m-%d")
        end_year =  datetime.datetime.strptime(date_find[-1].split(" ")[1], "%m/%d/%Y").strftime("%Y-%m-%d")

        ncol        = str(int(x_dim))
        nrow        = str(int(y_dim))
        return(  component_name + '__' + start_year + '_' 
              + end_year + '__' + nrow + '_by_' + ncol + '.nc' )

    # some other functions to post process stuff

    def ncdump(nc_fid):
        '''ncdump outputs dimensions, variables and their attribute information of netCDF4 files'''
        nc_attrs = nc_fid.ncattrs()
        nc_dims = [dim for dim in nc_fid.dimensions]  
        nc_vars = [var for var in nc_fid.variables] 
        return nc_attrs, nc_dims, nc_vars

    def writeArrayToArcGrid(arr,filename,xll,yll,cellsize,no_data_val):
        """ this takes a 2d numpy array and turns it into an .asc file """
        arr                = np.copy(arr)
        arr[np.isnan(arr)] = no_data_val
        headerstring       = bytes('NCOLS %d\nNROWS %d\nXLLCENTER %f\nYLLCENTER %f\nCELLSIZE %f\nNODATA_value %f\n' % 
            (arr.shape[1], arr.shape[0], xll, yll, cellsize, no_data_val), 'UTF-8')

        with open(filename,'wb') as fout:
            fout.write(headerstring)
            np.savetxt(fout,arr,'%5.2f')



    # Post process the files again, this time with the MFR added 
    outspace = os.path.join(output_folder, 'post_prcessed_with_MFR')
    if not os.path.exists(outspace):
        os.makedirs(outspace)

    # post process the whole model domain      
    os.chdir(os.path.join(output_folder))  # difficulty in making the path to the file so need to change into the output directory
    var = []; tot = []; nclist = []

    # Step 1: make list of files that you wish to process 
    for i in Desired_files:
        Da_file = create_file_reference(i)
        nclist.append(Da_file)

    # Step 2 average the daily dimension (len(t) is # of days in the run) to get a frame of annual stats
    for i, f in enumerate(nclist):
        nc_data = nc.Dataset(nclist[i])
        nc_attrs, nc_dims, nc_vars = ncdump(nc_data)
        nc_var = nc_vars[3]
        t = nc_data.variables['time'][:]
        y = nc_data.variables['y'][:]
        x = nc_data.variables['x'][:]
        nt = len(t)
        nrow = len(y)
        ncol = len(x)
        rd = np.zeros((nrow, ncol))  # create 0 array of the proper shape
        for day in range(nt):
            r_temp = nc_data.variables[str(nc_var)][day, :, :]
            r_filled = np.ma.filled(r_temp, fill_value=0)    # fills in missing values with 0s (i think) 
            rd = rd+r_filled                                 # sequentially add each day's value in each cell to the empty frame  
        r = rd/nt*365 # to create a one year average from all the years in model.  if want to add leap years add 0.25 

        # step 3: write each yearly average array to a .asc file
        keyname = Desired_files[i] 
        writeArrayToArcGrid(r, os.path.join(outspace, "{}_annual.asc".format(keyname)), XLLCORNER, YLLCORNER, CELLSIZE, -999)

        # Step 4: calculate total amounts of water in cubic meters per day and create statistics dataframe
        m3pd = ((cel_size**2)*r.sum()*.0254)/365 
        # print("{} total  {} [m3/d]".format(keyname, '%.1f' % m3pd))
        var.append(keyname) ; tot.append(m3pd)     # make lists to populate pandas dataframe

        nc_data.close()          # make sure to close the nc file so it doesnt stay open

    stat_frame = pd.DataFrame({'Variable' : var, 'total_[m3pd]': tot})    #in case you want the max and min#, "Max_[in]": mx, "Min_[in]":mn})
    stat_frame["total_[MGD]"] = stat_frame["total_[m3pd]"]/3785.41178       # put things in MGD if interested, 3785.41178 is number of gal in m3   
    stat_frame["total_[Mld]"] =stat_frame["total_[MGD]"]*3.78541            # Convert to mld
    Precip = list(stat_frame[stat_frame['Variable'] == 'rainfall']['total_[m3pd]'])[0]   # define the amount of calculated Precip
    Dir_net_inf = list(stat_frame[stat_frame['Variable'] == 'direct_net_infiltation']['total_[m3pd]'])[0]   # define the amount of calculated infiltration
    WB_ins = Precip + Dir_net_inf
    stat_frame['pct_of_pcip'] = stat_frame["total_[m3pd]"]/WB_ins
    stat_frame.to_csv(os.path.join(outspace, "IslandWide_Summary_Stats.csv"))

    # how does the model balance? 
    print("WATER BALANCE ratio: outs over ins water budget balanece =  {} % ".format(stat_frame['pct_of_pcip'].sum()-1))   # check water balance

    os.chdir(os.path.join("..", 'run'))  # then back out to the home directory
    
    
    # Step 2.2 average the daily dimension over months to get a frame of annual stats

    
    
    
    
    

    # calculate statistics for individual watersheds
    # note, for some reason will not overwrite csvs need to clear them out or recode to make this issue not an issue
    #create workspace
    outspace_table = os.path.join(output_folder, 'post_prcessed_with_MFR', "tables")
    if not os.path.exists(outspace_table):
        os.makedirs(outspace_table)
    sheds = (os.path.join(GIS_FOLDER, 'Watersheds\\Runoff_zones_sheds_WGS2S_clip.shp'))

    # process each raster layer into a table
    for i in (os.listdir(outspace)):
        if i.endswith('.asc'):
            outZSaT = ZonalStatisticsAsTable(sheds, "SHED_NAME", os.path.join(outspace, i), os.path.join(outspace_table, "temptab.dbf"))  # in arc format
            arcpy.TableToTable_conversion(outZSaT, outspace_table, "Table_{}_1.csv".format(i))                                            # take table out of stupid arc format and put into csv format 

    # this block takes each of the csvs, reads them and calculates water volumnes (m3/d) for each watershed
    templist = []
    for c in (os.listdir(os.path.join(outspace, "tables"))):
        if c.endswith('.csv'):
            data = pd.read_csv(os.path.join(outspace, "tables", c))
            keyname = c.split("Table_")[1].split("_annual")[0]                   # parameter being worked on
            data[keyname] = (data['MEAN']*.0254/365) * data['AREA'] 
            temp_frame = data[["SHED_NAME", keyname]]
            templist.append(temp_frame)

    summarry_frame1 = data[['SHED_NAME']]                                        # this is just sticking them all together into one dataframe
    for i in templist:
        summarry_frame1 = summarry_frame1.merge(i, on ='SHED_NAME', how='outer')


    # that was in actual volumns, now to convert each component into a fraction of the rainfall...
    templist2 = []
    for i in summarry_frame1.columns[1:]:
        temp_frame = data[['SHED_NAME']] ; temp_frame[i.split("-")[0]] = summarry_frame1[i]/summarry_frame1['rainfall']
        templist2.append(temp_frame)

    summarry_frame3 = data[['SHED_NAME']]
    for i in templist2:
        summarry_frame3 = summarry_frame3.merge(i, on ='SHED_NAME', how='outer')

    # other units here
    summarry_frame_4000 = summarry_frame1.set_index('SHED_NAME')
    summarry_frame_4 = summarry_frame_4000.select_dtypes(exclude=['object'])/3785.41178    # convert to million gallons per day
    
    summarry_frame_5 = summarry_frame_4.select_dtypes(exclude=['object'])*3.78541              # convert to million liters per day 

    summarry_frame1.to_csv(os.path.join(outspace, "watershed_summary_stats_volume_m3pd.csv"))
    summarry_frame3.to_csv(os.path.join(outspace, "watershed_summary_stats_percentages.csv"))
    summarry_frame_4.to_csv(os.path.join(outspace, "watershed_summary_stats_volumes_MGD.csv"))
    summarry_frame_5.to_csv(os.path.join(outspace, "watershed_summary_stats_volumes_Mld.csv"))
    
    
    return summarry_frame1, summarry_frame_5


In [17]:
# BASE CASE: PROCESSIIISIS it 

Out_fold = os.path.join("..", "1_Original_run_out")
if not os.path.exists(Out_fold):
    os.makedirs(Out_fold)
    
Control_File_Name = '1_Tutuila_cal_controlFile.ctl'
    
# changes the cell size in the ctrl file to whatever the rainfall adjustment grid size is
x_dim, y_dim, cel_size = mod_ctrl_file_cellSize(Control_File_Name)

# RUN the model
#Summary_frame_1, Summary_frame_5 = PP_model(Control_File_Name, Out_fold, x_dim, y_dim, Direct_infiltration_input_folder = "Direct_infiltration")


array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [18]:
# Run as non-funciton   def PP_model(Control_File_Name, output_folder, x_dim, y_dim, Direct_infiltration_input_folder):

# BASE CASE: PRESENT DAY, HIGH RESOLUTION

output_folder = os.path.join("..", "1_Original_run_out")
if not os.path.exists(Out_fold):
    os.makedirs(Out_fold)
    
# changes the cell size in the ctrl file to whatever the rainfall adjustment grid size is
x_dim, y_dim, cel_size = mod_ctrl_file_cellSize(Control_File_Name)



# Parameters
Desired_files = ['actual_et',  'direct_net_infiltation', 'direct_soil_moisture',
             'interception', 'net_infiltration', 'rainfall', 'runoff'] # 'delta_soil_storage',  'irrigation', 
XLLCORNER =      515000.000
YLLCORNER =      8409000.000
CELLSIZE  =      cel_size

# functions
def create_file_reference( component_name ):
    '''
    This is a simple convenience function that will form a path and filename to a
    given water budget component
    '''
    # specify the prefix, path to SWB2 output, timeframe, and resolution
    #output_path = os.path.join(os.getcwd(), "output")
    #prefix      = '\\'

# pull start and end dates from control file
    with open(os.path.join('.', Control_File_Name), 'r') as fin:   # open file 
        date_find = fin.read().splitlines(True)
    start_year = datetime.datetime.strptime(date_find[-2].split(" ")[1], "%m/%d/%Y").strftime("%Y-%m-%d")
    end_year =  datetime.datetime.strptime(date_find[-1].split(" ")[1], "%m/%d/%Y").strftime("%Y-%m-%d")

    ncol        = str(int(x_dim))
    nrow        = str(int(y_dim))
    return(  component_name + '__' + start_year + '_' 
          + end_year + '__' + nrow + '_by_' + ncol + '.nc' )

# some other functions to post process stuff

def ncdump(nc_fid):
    '''ncdump outputs dimensions, variables and their attribute information of netCDF4 files'''
    nc_attrs = nc_fid.ncattrs()
    nc_dims = [dim for dim in nc_fid.dimensions]  
    nc_vars = [var for var in nc_fid.variables] 
    return nc_attrs, nc_dims, nc_vars

def writeArrayToArcGrid(arr,filename,xll,yll,cellsize,no_data_val):
    """ this takes a 2d numpy array and turns it into an .asc file """
    arr                = np.copy(arr)
    arr[np.isnan(arr)] = no_data_val
    headerstring       = bytes('NCOLS %d\nNROWS %d\nXLLCENTER %f\nYLLCENTER %f\nCELLSIZE %f\nNODATA_value %f\n' % 
        (arr.shape[1], arr.shape[0], xll, yll, cellsize, no_data_val), 'UTF-8')

    with open(filename,'wb') as fout:
        fout.write(headerstring)
        np.savetxt(fout,arr,'%5.2f')



# Post process the files again, this time with the MFR added 
outspace = os.path.join(output_folder, 'post_prcessed_with_MFR')
if not os.path.exists(outspace):
    os.makedirs(outspace)

# post process the whole model domain      
# os.chdir(os.path.join(output_folder))  # difficulty in making the path to the file so need to change into the output directory
var = []; tot = []; nclist = []

# Step 1: make list of files that you wish to process 
for i in Desired_files:
    Da_file = create_file_reference(i)
    nclist.append(os.path.join(output_folder, Da_file))

# Step 2 average the daily dimension (len(t) is # of days in the run) to get a frame of annual stats
for i, f in enumerate(nclist):
    nc_data = nc.Dataset(nclist[i])
    nc_attrs, nc_dims, nc_vars = ncdump(nc_data)
    nc_var = nc_vars[3]
    t = nc_data.variables['time'][:]
    y = nc_data.variables['y'][:]
    x = nc_data.variables['x'][:]
    nt = len(t)
    nrow = len(y)
    ncol = len(x)
    rd = np.zeros((nrow, ncol))  # create 0 array of the proper shape
    for day in range(nt):
        r_temp = nc_data.variables[str(nc_var)][day, :, :]
        r_filled = np.ma.filled(r_temp, fill_value=0)    # fills in missing values with 0s (i think) 
        rd = rd+r_filled                                 # sequentially add each day's value in each cell to the empty frame  
    r = rd/nt*365 # to create a one year average from all the years in model.  if want to add leap years add 0.25 

    # step 3: write each yearly average array to a .asc file
    keyname = Desired_files[i] 
    writeArrayToArcGrid(r, os.path.join(outspace, "{}_annual.asc".format(keyname)), XLLCORNER, YLLCORNER, CELLSIZE, -999)

    # Step 4: calculate total amounts of water in cubic meters per day and create statistics dataframe
    m3pd = ((cel_size**2)*r.sum()*.0254)/365 
    # print("{} total  {} [m3/d]".format(keyname, '%.1f' % m3pd))
    var.append(keyname) ; tot.append(m3pd)     # make lists to populate pandas dataframe

    nc_data.close()          # make sure to close the nc file so it doesnt stay open

stat_frame = pd.DataFrame({'Variable' : var, 'total_[m3pd]': tot})    #in case you want the max and min#, "Max_[in]": mx, "Min_[in]":mn})
stat_frame["total_[MGD]"] = stat_frame["total_[m3pd]"]/3785.41178       # put things in MGD if interested, 3785.41178 is number of gal in m3   
stat_frame["total_[Mld]"] =stat_frame["total_[MGD]"]*3.78541            # Convert to mld
Precip = list(stat_frame[stat_frame['Variable'] == 'rainfall']['total_[m3pd]'])[0]   # define the amount of calculated Precip
Dir_net_inf = list(stat_frame[stat_frame['Variable'] == 'direct_net_infiltation']['total_[m3pd]'])[0]   # define the amount of calculated infiltration
WB_ins = Precip + Dir_net_inf
stat_frame['pct_of_pcip'] = stat_frame["total_[m3pd]"]/WB_ins
stat_frame.to_csv(os.path.join(outspace, "IslandWide_Summary_Stats.csv"))

# how does the model balance? 
print("WATER BALANCE ratio: outs over ins water budget balanece =  {} % ".format(stat_frame['pct_of_pcip'].sum()-1))   # check water balance

#os.chdir(os.path.join("..", 'run'))  # then back out to the home directory


# Step 2.2 average the daily dimension over months to get a frame of annual stats







# calculate statistics for individual watersheds
# note, for some reason will not overwrite csvs need to clear them out or recode to make this issue not an issue
#create workspace
outspace_table = os.path.join(output_folder, 'post_prcessed_with_MFR', "tables")
if not os.path.exists(outspace_table):
    os.makedirs(outspace_table)
sheds = (os.path.join(GIS_FOLDER, 'Watersheds\\Runoff_zones_sheds_WGS2S_clip.shp'))

# process each raster layer into a table
for i in (os.listdir(outspace)):
    if i.endswith('.asc'):
        outZSaT = ZonalStatisticsAsTable(sheds, "SHED_NAME", os.path.join(outspace, i), os.path.join(outspace_table, "temptab.dbf"))  # in arc format
        arcpy.TableToTable_conversion(outZSaT, outspace_table, "Table_{}_1.csv".format(i))                                            # take table out of stupid arc format and put into csv format 

# this block takes each of the csvs, reads them and calculates water volumnes (m3/d) for each watershed
templist = []
for c in (os.listdir(os.path.join(outspace, "tables"))):
    if c.endswith('.csv'):
        data = pd.read_csv(os.path.join(outspace, "tables", c))
        keyname = c.split("Table_")[1].split("_annual")[0]                   # parameter being worked on
        data[keyname] = (data['MEAN']*.0254/365) * data['AREA'] 
        temp_frame = data[["SHED_NAME", keyname]]
        templist.append(temp_frame)

summarry_frame1 = data[['SHED_NAME']]                                        # this is just sticking them all together into one dataframe
for i in templist:
    summarry_frame1 = summarry_frame1.merge(i, on ='SHED_NAME', how='outer')


# that was in actual volumns, now to convert each component into a fraction of the rainfall...
templist2 = []
for i in summarry_frame1.columns[1:]:
    temp_frame = data[['SHED_NAME']] ; temp_frame[i.split("-")[0]] = summarry_frame1[i]/summarry_frame1['rainfall']
    templist2.append(temp_frame)

summarry_frame3 = data[['SHED_NAME']]
for i in templist2:
    summarry_frame3 = summarry_frame3.merge(i, on ='SHED_NAME', how='outer')

# other units here
summarry_frame_4000 = summarry_frame1.set_index('SHED_NAME')
summarry_frame_4 = summarry_frame_4000.select_dtypes(exclude=['object'])/3785.41178    # convert to million gallons per day

summarry_frame_5 = summarry_frame_4.select_dtypes(exclude=['object'])*3.78541              # convert to million liters per day 

summarry_frame1.to_csv(os.path.join(outspace, "watershed_summary_stats_volume_m3pd.csv"))
summarry_frame3.to_csv(os.path.join(outspace, "watershed_summary_stats_percentages.csv"))
summarry_frame_4.to_csv(os.path.join(outspace, "watershed_summary_stats_volumes_MGD.csv"))
summarry_frame_5.to_csv(os.path.join(outspace, "watershed_summary_stats_volumes_Mld.csv"))

WATER BALANCE ratio: outs over ins water budget balanece =  1.0036519123027903 % 


In [19]:
# changes the cell size in the ctrl file to whatever the rainfall adjustment grid size is
x_dim, y_dim, cel_size = mod_ctrl_file_cellSize(Control_File_Name)



# Parameters
Desired_files = ['actual_et',  'direct_net_infiltation', 'direct_soil_moisture',
             'interception', 'net_infiltration', 'rainfall', 'runoff'] # 'delta_soil_storage',  'irrigation', 
XLLCORNER =      515000.000
YLLCORNER =      8409000.000
CELLSIZE  =      cel_size

for i in Desired_files:
    Da_file = create_file_reference(i)
    nclist.append(os.path.join(output_folder, Da_file))

In [44]:
# Calculate MONTHLY island wide statistics 
outspace_mon = os.path.join(output_folder, 'post_prcessed_with_MFR', "Monthly_summaries")
if not os.path.exists(outspace_mon):
    os.makedirs(outspace_mon)

results_frame = pd.DataFrame()

# Read in the daily dimension (len(t) is # of days in the run) 
for i, f in enumerate(nclist):                      # loop over each water budget component
    nc_data = nc.Dataset(nclist[i])                 # read in netcdef file 
    nc_attrs, nc_dims, nc_vars = ncdump(nc_data)    # grab dimension/variable headers
    nc_var = nc_vars[3]                             # the variabel of interest
    t = nc_data.variables['time'][:]
    y = nc_data.variables['y'][:]
    x = nc_data.variables['x'][:]
    nrow = len(y)
    ncol = len(x)
    
    # day numbers from the 1 year netcdf dataset that correspond to each month.  Will probably go bad in model is run longer
    Juli_months = {"JAN":t[0:31],  "FEB":t[31:59],  "MAR":t[59:90],  "APR":t[90:120 ],  "MAY":t[120:151 ],
                    "JUN":t[151:181 ],  "JUL":t[181:212 ],  "AUG":t[212:243 ],  
                                "SEP":t[243:273 ],  "OCT":t[273:304 ],  "NOV":t[304:334 ],  "DEC":t[334:365]}
    
    for month in Juli_months.keys():
           
        dest_array = np.zeros((nrow, ncol))  # create 0 array of the proper shape
        nt = len(Juli_months[month])         # length of the month
        
        for day in Juli_months[month]:
            arr_temp = nc_data.variables[str(nc_var)][day, :, :]
            arr_filled = np.ma.filled(arr_temp, fill_value=0)    # fills in missing values with 0s (i think) 
            dest_array = dest_array+arr_filled                                 # sequentially add each day's value in each cell to the empty frame  
        fin_array = dest_array/nt*nt # to create the daily average value from all the years in model.  if want to add leap years add 0.25 

        # step 3: write each yearly average array to a .asc file 
        writeArrayToArcGrid(fin_array, os.path.join(outspace_mon, "{}_monthly_{}.asc".format(nc_var, month)), XLLCORNER, YLLCORNER, CELLSIZE, -999)

        # Step 4: calculate total amounts of water in cubic meters per day and create statistics dataframe
        m3pd = ((cel_size**2)*fin_array.sum()*.0254)/nt 
        #print("{}_{} total  {} [m3/d]".format(month, nc_var, '%.1f' % m3pd))
        
        results_frame.loc[month, nc_var] = m3pd
        results_frame.to_csv(os.path.join(outspace, "Monthly_summary_m3pd.csv"))
        
        results_frame_MGD = results_frame*(264.172/1000000)
        results_frame_MGD.to_csv(os.path.join(outspace, "Monthly_summary_MGD.csv"))
        
        results_frame_Mld = results_frame_MGD*3.78541 
        results_frame_Mld.to_csv(os.path.join(outspace, "Monthly_summary_Mld.csv"))
        
    nc_data.close()          # make sure to close the nc file so it doesnt stay open

JAN_actual_et total  258190.3 [m3/d]
FEB_actual_et total  242331.0 [m3/d]
MAR_actual_et total  226697.3 [m3/d]
APR_actual_et total  208297.3 [m3/d]
MAY_actual_et total  209316.1 [m3/d]
JUN_actual_et total  195189.5 [m3/d]
JUL_actual_et total  186486.9 [m3/d]
AUG_actual_et total  217705.9 [m3/d]
SEP_actual_et total  237013.2 [m3/d]
OCT_actual_et total  247152.9 [m3/d]
NOV_actual_et total  261529.3 [m3/d]
DEC_actual_et total  251368.6 [m3/d]
JAN_direct_net_infiltation total  57274.7 [m3/d]
FEB_direct_net_infiltation total  57274.7 [m3/d]
MAR_direct_net_infiltation total  57274.7 [m3/d]
APR_direct_net_infiltation total  57274.7 [m3/d]
MAY_direct_net_infiltation total  57274.7 [m3/d]
JUN_direct_net_infiltation total  57274.7 [m3/d]
JUL_direct_net_infiltation total  57274.7 [m3/d]
AUG_direct_net_infiltation total  57274.7 [m3/d]
SEP_direct_net_infiltation total  57274.7 [m3/d]
OCT_direct_net_infiltation total  57274.7 [m3/d]
NOV_direct_net_infiltation total  57274.7 [m3/d]
DEC_direct_net_in

In [65]:
np.set_printoptions(threshold=30)

In [71]:
np.shape(dest_array)

(400, 700)

In [74]:
dest_array[200]

array([  0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.  

In [54]:
nc_data = nc.Dataset(nclist[i])                 # read in netcdef file 

# Run all the Scenarios
Each scenario has it's own cell

In [4]:
# BASE CASE: PRESENT DAY, HIGH RESOLUTION

Out_fold = os.path.join("..", "1_Original_run_out")
if not os.path.exists(Out_fold):
    os.makedirs(Out_fold)
    
Control_File_Name = '1_Tutuila_cal_controlFile.ctl'
    
# changes the cell size in the ctrl file to whatever the rainfall adjustment grid size is
x_dim, y_dim, cel_size = mod_ctrl_file_cellSize(Control_File_Name)

# RUN the model
Summary_frame_1 = run_model(Control_File_Name, Out_fold, x_dim, y_dim, Direct_infiltration_input_folder = "Direct_infiltration")


1_Tutuila_cal_controlFile.ctl
Run model 1, no MFR
WATER BALANCE ratio: outs over ins water budget balanece =  1.0036984073525663 % 
MFR total in MGD is 6.837100648803507
Caclculating MFR
Run model 2, with MFR
WATER BALANCE ratio: outs over ins water budget balanece =  1.0036519123027903 % 


In [ ]:
# Scenario 1  PRESENT DAY just for comparison to base case  **

""" CHANGES: 
LAND_USE ARC_GRID ../input/Land_use_Soils_data/lu_grid.asc   to
LAND_USE ARC_GRID ../input/Land_use_Soils_data_Scen1/lu_grid.asc

Be sure to use: 
Direct_infiltration_input_folder = "Direct_infiltration_Scen_1"  """


Out_fold = os.path.join("..", "2_scn1_noclimate_out")
if not os.path.exists(Out_fold):
    os.makedirs(Out_fold)
    
Control_File_Name = '2_scn1_noclimate_controlFile.ctl'
    
# changes the cell size in the ctrl file to whatever the rainfall adjustment grid size is
x_dim, y_dim, cel_size = mod_ctrl_file_cellSize(Control_File_Name)

# RUN the model
Summary_frame_1 = run_model(Control_File_Name, Out_fold, x_dim, y_dim, Direct_infiltration_input_folder = "Direct_infiltration_Scen_1")

In [ ]:
# Scenario 2  PRESENT DAY just for comparison to base case  **

""" CHANGES: 
LAND_USE ARC_GRID ../input/Land_use_Soils_data/lu_grid.asc   to
LAND_USE ARC_GRID ../input/Land_use_Soils_data_Scen2/lu_grid.asc

Be sure to use: 
Direct_infiltration_input_folder = "Direct_infiltration_Scen_2"   """


Out_fold = os.path.join("..", "3_scn2_noclimate_out")
if not os.path.exists(Out_fold):
    os.makedirs(Out_fold)
    
Control_File_Name = '3_scn2_noclimate_controlFile.ctl'
    
# changes the cell size in the ctrl file to whatever the rainfall adjustment grid size is
x_dim, y_dim, cel_size = mod_ctrl_file_cellSize(Control_File_Name)

# RUN the model
Summary_frame_1 = run_model(Control_File_Name, Out_fold, x_dim, y_dim, Direct_infiltration_input_folder = "Direct_infiltration_Scen_2")

In [7]:
# Scenario 3  PRESENT DAY just for comparison to base case **

""" CHANGES: 
LAND_USE ARC_GRID ../input/Land_use_Soils_data/lu_grid.asc   to
LAND_USE ARC_GRID ../input/Land_use_Soils_data_Scen3/lu_grid.asc

Be sure to use: 
Direct_infiltration_input_folder = "Direct_infiltration_Scen_3"   """

Out_fold = os.path.join("..", "4_scn3_noclimate_out")
if not os.path.exists(Out_fold):
    os.makedirs(Out_fold)
    
Control_File_Name = '4_scn3_noclimate_controlFile.ctl'
    
# changes the cell size in the ctrl file to whatever the rainfall adjustment grid size is
x_dim, y_dim, cel_size = mod_ctrl_file_cellSize(Control_File_Name)

# RUN the model
Summary_frame_1 = run_model(Control_File_Name, Out_fold, x_dim, y_dim, Direct_infiltration_input_folder = "Direct_infiltration_Scen_3")

4_scn3_noclimate_controlFile.ctl
Run model 1, no MFR
WATER BALANCE ratio: outs over ins water budget balanece =  1.0092252500891026 % 
MFR total in MGD is 7.161306119793166
Caclculating MFR
Run model 2, with MFR
WATER BALANCE ratio: outs over ins water budget balanece =  1.0091042641826515 % 


In [8]:
# BASE CASE FUTURE PRECIP LOW RESOLUTION

Out_fold = os.path.join("..", "5_FutureClimate_BASE_out")
if not os.path.exists(Out_fold):
    os.makedirs(Out_fold)
    
    
""" CHANGES: 

PRECIPITATION ARC_GRID ../input/Gridded_rain/prism_ppt_tutuila_30yr_normal_%0m.asc    to 
PRECIPITATION ARC_GRID ../input/Gridded_rain_RCP_Present/cchanged_ppt_present_month_%m.asc

"""
    
Control_File_Name = '5_FutureClimate_BASE_controlFile.ctl'
    
# changes the cell size in the ctrl file to whatever the rainfall adjustment grid size is
x_dim, y_dim, cel_size = mod_ctrl_file_cellSize(Control_File_Name)

# RUN the model
Summary_frame_1 = run_model(Control_File_Name, Out_fold, x_dim, y_dim, Direct_infiltration_input_folder = "Direct_infiltration")


5_FutureClimate_BASE_controlFile.ctl
Run model 1, no MFR
WATER BALANCE ratio: outs over ins water budget balanece =  0.9935863969904517 % 
MFR total in MGD is 1.6262156303707644
Caclculating MFR
Run model 2, with MFR
WATER BALANCE ratio: outs over ins water budget balanece =  0.9936192571467952 % 


In [9]:
# RCP4.5 FUTURE no LU change

Out_fold = os.path.join("..", "6_FutureClimate_RCP45_out")
if not os.path.exists(Out_fold):
    os.makedirs(Out_fold)
    

""" CHANGES from CC base: 

PRECIPITATION ARC_GRID ../input/Gridded_rain_RCP_Present/cchanged_ppt_present_month_%m.asc   to 
PRECIPITATION ARC_GRID ../input/Gridded_rain_RCP_RCP4p5/cchanged_ppt_rcp45_month_%m.asc 

"""
    
Control_File_Name = '6_FutureClimate_RCP45_controlFile.ctl'


# changes the cell size in the ctrl file to whatever the rainfall adjustment grid size is
x_dim, y_dim, cel_size = mod_ctrl_file_cellSize(Control_File_Name)

# RUN the model
Summary_frame_1 = run_model(Control_File_Name, Out_fold, x_dim, y_dim, Direct_infiltration_input_folder = "Direct_infiltration")
 
    

6_FutureClimate_RCP45_controlFile.ctl
Run model 1, no MFR
WATER BALANCE ratio: outs over ins water budget balanece =  0.9952827476352679 % 
MFR total in MGD is 2.686310263404302
Caclculating MFR
Run model 2, with MFR
WATER BALANCE ratio: outs over ins water budget balanece =  0.9953167683324209 % 


In [10]:
# RCP8.5 FUTURE no LU change

Out_fold = os.path.join("..", "7_FutureClimate_RCP85_out")
if not os.path.exists(Out_fold):
    os.makedirs(Out_fold)
    
""" CHANGES from CC base: 

PRECIPITATION ARC_GRID ../input/Gridded_rain_RCP_Present/cchanged_ppt_present_month_%m.asc   to 
PRECIPITATION ARC_GRID ../input/Gridded_rain_RCP_RCP4p5/cchanged_ppt_rcp85_month_%m.asc 

"""
    
Control_File_Name = '7_FutureClimate_RCP85_controlFile.ctl'


# changes the cell size in the ctrl file to whatever the rainfall adjustment grid size is
x_dim, y_dim, cel_size = mod_ctrl_file_cellSize(Control_File_Name)

# RUN the model
Summary_frame_1 = run_model(Control_File_Name, Out_fold, x_dim, y_dim, Direct_infiltration_input_folder = "Direct_infiltration")
 
    

7_FutureClimate_RCP85_controlFile.ctl
Run model 1, no MFR
WATER BALANCE ratio: outs over ins water budget balanece =  0.9954957978195154 % 
MFR total in MGD is 2.4339446716939075
Caclculating MFR
Run model 2, with MFR
WATER BALANCE ratio: outs over ins water budget balanece =  0.9955270102503937 % 


In [11]:
# Present FUTURE PRECIP Scn1 LU  **

Out_fold = os.path.join("..", "8_Future_Present_Scn1_out")
if not os.path.exists(Out_fold):
    os.makedirs(Out_fold)
    
""" CHANGES: 

PRECIPITATION ARC_GRID ../input/Gridded_rain/prism_ppt_tutuila_30yr_normal_%0m.asc    to 
PRECIPITATION ARC_GRID ../input/Gridded_rain_RCP_Present/cchanged_ppt_present_month_%m.asc

LAND_USE ARC_GRID ../input/Land_use_Soils_data/lu_grid.asc   to
LAND_USE ARC_GRID ../input/Land_use_Soils_data_Scen1/lu_grid.asc

Be sure to use: 
Direct_infiltration_input_folder = "Direct_infiltration_Scen_1"

"""   

Control_File_Name = '8_Future_Present_Scn1_controlFile.ctl'


# changes the cell size in the ctrl file to whatever the rainfall adjustment grid size is
x_dim, y_dim, cel_size = mod_ctrl_file_cellSize(Control_File_Name)

# RUN the model
Summary_frame_1 = run_model(Control_File_Name, Out_fold, x_dim, y_dim, Direct_infiltration_input_folder = "Direct_infiltration_Scen_1")


8_Future_Present_Scn1_controlFile.ctl
Run model 1, no MFR
WATER BALANCE ratio: outs over ins water budget balanece =  1.0049894159138755 % 
MFR total in MGD is 1.709070415153855
Caclculating MFR
Run model 2, with MFR
WATER BALANCE ratio: outs over ins water budget balanece =  1.0049627924433389 % 


In [12]:
# Present FUTURE PRECIP Scn2 LU  **

Out_fold = os.path.join("..", "9_Future_Present_Scn2_out")
if not os.path.exists(Out_fold):
    os.makedirs(Out_fold)
    
    
""" CHANGES: 

PRECIPITATION ARC_GRID ../input/Gridded_rain/prism_ppt_tutuila_30yr_normal_%0m.asc    to 
PRECIPITATION ARC_GRID ../input/Gridded_rain_RCP_Present/cchanged_ppt_present_month_%m.asc

LAND_USE ARC_GRID ../input/Land_use_Soils_data/lu_grid.asc   to
LAND_USE ARC_GRID ../input/Land_use_Soils_data_Scen2/lu_grid.asc

Be sure to use: 
Direct_infiltration_input_folder = "Direct_infiltration_Scen_2"

"""   

Control_File_Name = '9_Future_Present_Scn2_controlFile.ctl'


# changes the cell size in the ctrl file to whatever the rainfall adjustment grid size is
x_dim, y_dim, cel_size = mod_ctrl_file_cellSize(Control_File_Name)

# RUN the model
Summary_frame_1 = run_model(Control_File_Name, Out_fold, x_dim, y_dim, Direct_infiltration_input_folder = "Direct_infiltration_Scen_2")


9_Future_Present_Scn2_controlFile.ctl
Run model 1, no MFR
WATER BALANCE ratio: outs over ins water budget balanece =  0.989130932432899 % 
MFR total in MGD is 1.6054147258726712
Caclculating MFR
Run model 2, with MFR
WATER BALANCE ratio: outs over ins water budget balanece =  0.9891860749155397 % 


In [13]:
# Present FUTURE PRECIP Scn3 LU  **

Out_fold = os.path.join("..", "10_Future_Present_Scn3_out")
if not os.path.exists(Out_fold):
    os.makedirs(Out_fold)

    
        
""" CHANGES: 

PRECIPITATION ARC_GRID ../input/Gridded_rain/prism_ppt_tutuila_30yr_normal_%0m.asc    to 
PRECIPITATION ARC_GRID ../input/Gridded_rain_RCP_Present/cchanged_ppt_present_month_%m.asc

LAND_USE ARC_GRID ../input/Land_use_Soils_data/lu_grid.asc   to
LAND_USE ARC_GRID ../input/Land_use_Soils_data_Scen3/lu_grid.asc

Be sure to use: 
Direct_infiltration_input_folder = "Direct_infiltration_Scen_3"

"""   

Control_File_Name = '10_Future_Present_Scn3_controlFile.ctl'


# changes the cell size in the ctrl file to whatever the rainfall adjustment grid size is
x_dim, y_dim, cel_size = mod_ctrl_file_cellSize(Control_File_Name)

# RUN the model
Summary_frame_1 = run_model(Control_File_Name, Out_fold, x_dim, y_dim, Direct_infiltration_input_folder = "Direct_infiltration_Scen_3")


10_Future_Present_Scn3_controlFile.ctl
Run model 1, no MFR
WATER BALANCE ratio: outs over ins water budget balanece =  1.0015907070507453 % 
MFR total in MGD is 1.7335346858895908
Caclculating MFR
Run model 2, with MFR
WATER BALANCE ratio: outs over ins water budget balanece =  1.0015820730664524 % 


In [14]:
# RCP4.5 FUTURE PRECIP Scn1 LU

Out_fold = os.path.join("..", "11_Future_RCP4.5_Scn1_out")
if not os.path.exists(Out_fold):
    os.makedirs(Out_fold)
    
    
""" CHANGES: 

PRECIPITATION ARC_GRID ../input/Gridded_rain/prism_ppt_tutuila_30yr_normal_%0m.asc    to 
PRECIPITATION ARC_GRID ../input/Gridded_rain_RCP_RCP4p5/CChanged_ppt_rcp45_month_%m.asc

LAND_USE ARC_GRID ../input/Land_use_Soils_data/lu_grid.asc   to
LAND_USE ARC_GRID ../input/Land_use_Soils_data_Scen1/lu_grid.asc

Be sure to use: 
Direct_infiltration_input_folder = "Direct_infiltration_Scen_1"

"""   

Control_File_Name = '11_Future_RCP4.5_Scn1_controlFile.ctl'


# changes the cell size in the ctrl file to whatever the rainfall adjustment grid size is
x_dim, y_dim, cel_size = mod_ctrl_file_cellSize(Control_File_Name)

# RUN the model
Summary_frame_1 = run_model(Control_File_Name, Out_fold, x_dim, y_dim, Direct_infiltration_input_folder = "Direct_infiltration_Scen_1")

11_Future_RCP4.5_Scn1_controlFile.ctl
Run model 1, no MFR
WATER BALANCE ratio: outs over ins water budget balanece =  1.0051347991756368 % 
MFR total in MGD is 2.8046980153802097
Caclculating MFR
Run model 2, with MFR
WATER BALANCE ratio: outs over ins water budget balanece =  1.0050964365984045 % 


In [15]:
# RCP4.5 FUTURE PRECIP Scn2 LU

Out_fold = os.path.join("..", "12_Future_RCP4.5_Scn2_out")
if not os.path.exists(Out_fold):
    os.makedirs(Out_fold)
    
    
""" CHANGES: 

PRECIPITATION ARC_GRID ../input/Gridded_rain/prism_ppt_tutuila_30yr_normal_%0m.asc    to 
PRECIPITATION ARC_GRID ../input/Gridded_rain_RCP_RCP4p5/CChanged_ppt_rcp45_month_%m.asc

LAND_USE ARC_GRID ../input/Land_use_Soils_data/lu_grid.asc   to
LAND_USE ARC_GRID ../input/Land_use_Soils_data_Scen2/lu_grid.asc

Be sure to use: 
Direct_infiltration_input_folder = "Direct_infiltration_Scen_2"

"""   

Control_File_Name = '12_Future_RCP4.5_Scn2_controlFile.ctl'


# changes the cell size in the ctrl file to whatever the rainfall adjustment grid size is
x_dim, y_dim, cel_size = mod_ctrl_file_cellSize(Control_File_Name)

# RUN the model
Summary_frame_1 = run_model(Control_File_Name, Out_fold, x_dim, y_dim, Direct_infiltration_input_folder = "Direct_infiltration_Scen_2")

12_Future_RCP4.5_Scn2_controlFile.ctl
Run model 1, no MFR
WATER BALANCE ratio: outs over ins water budget balanece =  0.9913753414171069 % 
MFR total in MGD is 2.653468184382694
Caclculating MFR
Run model 2, with MFR
WATER BALANCE ratio: outs over ins water budget balanece =  0.991436942501311 % 


In [16]:
# RCP4.5 FUTURE PRECIP Scn3 LU

Out_fold = os.path.join("..", "13_Future_RCP4.5_Scn3_out")
if not os.path.exists(Out_fold):
    os.makedirs(Out_fold)
    
    
""" CHANGES: 

PRECIPITATION ARC_GRID ../input/Gridded_rain/prism_ppt_tutuila_30yr_normal_%0m.asc    to 
PRECIPITATION ARC_GRID ../input/Gridded_rain_RCP_RCP4p5/CChanged_ppt_rcp45_month_%m.asc

LAND_USE ARC_GRID ../input/Land_use_Soils_data/lu_grid.asc   to
LAND_USE ARC_GRID ../input/Land_use_Soils_data_Scen3/lu_grid.asc

Be sure to use: 
Direct_infiltration_input_folder = "Direct_infiltration_Scen_3"

"""   

Control_File_Name = '13_Future_RCP4.5_Scn3_controlFile.ctl'


# changes the cell size in the ctrl file to whatever the rainfall adjustment grid size is
x_dim, y_dim, cel_size = mod_ctrl_file_cellSize(Control_File_Name)

# RUN the model
Summary_frame_1 = run_model(Control_File_Name, Out_fold, x_dim, y_dim, Direct_infiltration_input_folder = "Direct_infiltration_Scen_3")

13_Future_RCP4.5_Scn3_controlFile.ctl
Run model 1, no MFR
WATER BALANCE ratio: outs over ins water budget balanece =  1.0024127019314708 % 
MFR total in MGD is 2.846047966771728
Caclculating MFR
Run model 2, with MFR
WATER BALANCE ratio: outs over ins water budget balanece =  1.0023943672600382 % 


In [17]:
# RCP8.5 FUTURE PRECIP Scn1 LU

Out_fold = os.path.join("..", "14_Future_RCP8.5_Scn1_out")
if not os.path.exists(Out_fold):
    os.makedirs(Out_fold)
    
""" CHANGES: 

PRECIPITATION ARC_GRID ../input/Gridded_rain/prism_ppt_tutuila_30yr_normal_%0m.asc    to 
PRECIPITATION ARC_GRID ../input/Gridded_rain_RCP_RCP8p5/CChanged_ppt_rcp85_month_%m.asc

LAND_USE ARC_GRID ../input/Land_use_Soils_data/lu_grid.asc   to
LAND_USE ARC_GRID ../input/Land_use_Soils_data_Scen1/lu_grid.asc

Be sure to use: 
Direct_infiltration_input_folder = "Direct_infiltration_Scen_1"

"""   

Control_File_Name = '14_Future_RCP8.5_Scn1_controlFile.ctl'


# changes the cell size in the ctrl file to whatever the rainfall adjustment grid size is
x_dim, y_dim, cel_size = mod_ctrl_file_cellSize(Control_File_Name)

# RUN the model
Summary_frame_1 = run_model(Control_File_Name, Out_fold, x_dim, y_dim, Direct_infiltration_input_folder = "Direct_infiltration_Scen_1")

14_Future_RCP8.5_Scn1_controlFile.ctl
Run model 1, no MFR
WATER BALANCE ratio: outs over ins water budget balanece =  1.0059135769062446 % 
MFR total in MGD is 2.5416235888536605
Caclculating MFR
Run model 2, with MFR
WATER BALANCE ratio: outs over ins water budget balanece =  1.005871136998719 % 


In [18]:
# RCP8.5 FUTURE PRECIP Scn1 LU

Out_fold = os.path.join("..", "15_Future_RCP8.5_Scn2_out")
if not os.path.exists(Out_fold):
    os.makedirs(Out_fold)
    
""" CHANGES: 

PRECIPITATION ARC_GRID ../input/Gridded_rain/prism_ppt_tutuila_30yr_normal_%0m.asc    to 
PRECIPITATION ARC_GRID ../input/Gridded_rain_RCP_RCP8p5/CChanged_ppt_rcp85_month_%m.asc

LAND_USE ARC_GRID ../input/Land_use_Soils_data/lu_grid.asc   to
LAND_USE ARC_GRID ../input/Land_use_Soils_data_Scen2/lu_grid.asc

Be sure to use: 
Direct_infiltration_input_folder = "Direct_infiltration_Scen_2"

"""   

Control_File_Name = '15_Future_RCP8.5_Scn2_controlFile.ctl'


# changes the cell size in the ctrl file to whatever the rainfall adjustment grid size is
x_dim, y_dim, cel_size = mod_ctrl_file_cellSize(Control_File_Name)

# RUN the model
Summary_frame_1 = run_model(Control_File_Name, Out_fold, x_dim, y_dim, Direct_infiltration_input_folder = "Direct_infiltration_Scen_2")

15_Future_RCP8.5_Scn2_controlFile.ctl
Run model 1, no MFR
WATER BALANCE ratio: outs over ins water budget balanece =  0.991338738856921 % 
MFR total in MGD is 2.407160408105533
Caclculating MFR
Run model 2, with MFR
WATER BALANCE ratio: outs over ins water budget balanece =  0.9913982613090597 % 


In [19]:
# RCP8.5 FUTURE PRECIP Scn1 LU

Out_fold = os.path.join("..", "16_Future_RCP8.5_Scn3_out")
if not os.path.exists(Out_fold):
    os.makedirs(Out_fold)
    
""" CHANGES: 

PRECIPITATION ARC_GRID ../input/Gridded_rain/prism_ppt_tutuila_30yr_normal_%0m.asc    to 
PRECIPITATION ARC_GRID ../input/Gridded_rain_RCP_RCP8p5/CChanged_ppt_rcp85_month_%m.asc

LAND_USE ARC_GRID ../input/Land_use_Soils_data/lu_grid.asc   to
LAND_USE ARC_GRID ../input/Land_use_Soils_data_Scen3/lu_grid.asc

Be sure to use: 
Direct_infiltration_input_folder = "Direct_infiltration_Scen_3"

"""   

Control_File_Name = '16_Future_RCP8.5_Scn3_controlFile.ctl'


# changes the cell size in the ctrl file to whatever the rainfall adjustment grid size is
x_dim, y_dim, cel_size = mod_ctrl_file_cellSize(Control_File_Name)

# RUN the model
Summary_frame_1 = run_model(Control_File_Name, Out_fold, x_dim, y_dim, Direct_infiltration_input_folder = "Direct_infiltration_Scen_3")

16_Future_RCP8.5_Scn3_controlFile.ctl
Run model 1, no MFR
WATER BALANCE ratio: outs over ins water budget balanece =  1.0029100768006307 % 
MFR total in MGD is 2.5771864239838105
Caclculating MFR
Run model 2, with MFR
WATER BALANCE ratio: outs over ins water budget balanece =  1.002888846357401 % 
